# Understanding Memory in LLMs

In the previous Notebook 03, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. [Bing Chat](http://chat.bing.com/) is a search engine with a GPT-4 model that utilizes the content of search results to provide context and deliver accurate responses to queries.

However, we have yet to discover how to engage in a conversation with the LLM. With Bing Chat, this is possible, as the LLM can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

The aim of this Notebook is to demonstrate how we can "provide memory" to the LLM by utilizing prompts and context.

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory
from openai.error import OpenAIError
from langchain.docstore.document import Document
from langchain.memory import CosmosDBChatMessageHistory

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from app.utils import (
    get_search_results,
    order_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    embed_docs,
    search_docs,
    get_answer,
)

from app.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_CHAT_PROMPT

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"


# Change these below with your own services credentials
AZURE_SEARCH_ENDPOINT = "ENTER YOUR VALUE"
AZURE_SEARCH_KEY = "ENTER YOUR VALUE"  
AZURE_OPENAI_ENDPOINT= "ENTER YOUR VALUE"
AZURE_OPENAI_API_KEY = "ENTER YOUR VALUE"
AZURE_COSMOSDB_ENDPOINT = "ENTER YOUR VALUE"
AZURE_COSMOSDB_NAME = "ENTER YOUR VALUE"
AZURE_COSMOSDB_CONTAINER_NAME = "ENTER YOUR VALUE"
AZURE_COMOSDB_CONNECTION_STRING = "ENTER YOUR VALUE"


In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_COSMOSDB_ENDPOINT"]=  AZURE_COSMOSDB_ENDPOINT
os.environ["AZURE_COSMOSDB_NAME"]=  AZURE_COSMOSDB_NAME
os.environ["AZURE_COSMOSDB_CONTAINER_NAME"]=  AZURE_COSMOSDB_CONTAINER_NAME
os.environ["AZURE_COMOSDB_CONNECTION_STRING"]=  AZURE_COMOSDB_CONNECTION_STRING

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Can you rephrase what you just said?"

In [4]:
# Define model
MODEL = "gpt-35-turbo"
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5)

In [5]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Game playing: Reinforcement learning can be used to train game-playing agents to make optimal decisions in games like chess, Go, and poker.

2. Robotics: Reinforcement learning can be used to train robots to perform complex tasks like grasping objects, walking, and navigating through environments.

3. Advertising: Reinforcement learning can be used to optimize advertising campaigns by learning which ads perform best in different contexts and adjusting bids accordingly.

4. Finance: Reinforcement learning can be used to optimize trading strategies by learning which actions lead to the highest returns and adjusting trades accordingly.

5. Manufacturing: Reinforcement learning can be used to optimize manufacturing processes by learning which actions lead to the most efficient and cost-effective production.

6. Autonomous vehicles: Reinforcement learning can be used to train autonomous vehicles to make safe and efficient driving decisions in complex environments.

7. Healthcare: Reinforcement learning can be used to optimize treatment decisions for patients by learning which treatments lead to the best outcomes.

8. Energy management: Reinforcement learning can be used to optimize energy consumption in buildings and other systems by learning which actions lead to the most efficient use of resources.

In [7]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

'I apologize, but as an AI language model, I am not able to recall what I just said. Can you please provide me with the context or the sentence you want me to rephrase?'

As you can see, it doesn't remember what it just responded. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [10]:
chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION})

'Sure, here are some practical applications of reinforcement learning:\n\n1. Game playing: Reinforcement learning can train game-playing agents to make optimal decisions in games like chess, Go, and poker.\n\n2. Robotics: Reinforcement learning can teach robots to perform complex tasks like grasping objects, walking, and navigating through environments.\n\n3. Advertising: Reinforcement learning can optimize advertising campaigns by learning which ads perform best in different contexts and adjusting bids accordingly.\n\n4. Finance: Reinforcement learning can optimize trading strategies by learning which actions lead to the highest returns and adjusting trades accordingly.\n\n5. Manufacturing: Reinforcement learning can optimize manufacturing processes by learning which actions lead to the most efficient and cost-effective production.\n\n6. Autonomous vehicles: Reinforcement learning can train autonomous vehicles to make safe and efficient driving decisions in complex environments.\n\n7.

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In order to not duplicate code, we have put many of the code used in Notebook 3 into functions. These functions are in the app/utils.py and app/prompts.py files This way we can use these functios in the app that we will build later.

In [11]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

agg_search_results = get_search_results(QUESTION, indexes)
ordered_results = order_search_results(agg_search_results, reranker_threshold=1)

In [12]:
docs = []
for key,value in ordered_results.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

# Calculate number of tokens of our docs
tokens_limit = model_tokens_limit(MODEL)

if(len(docs)>0):
    num_tokens = num_tokens_from_docs(docs)
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")


Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 52495


In [13]:
%%time
if num_tokens > tokens_limit:
    index = embed_docs(docs)
    top_docs = search_docs(index,QUESTION)
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff or map_reduce
    
    num_tokens = num_tokens_from_docs(top_docs)   
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Number of chunks: 48


2023-05-24 17:08:07.256 INFO    faiss.loader: Loading faiss with AVX2 support.
2023-05-24 17:08:07.259 INFO    faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-05-24 17:08:07.260 INFO    faiss.loader: Loading faiss.
2023-05-24 17:08:07.296 INFO    faiss.loader: Successfully loaded faiss.


Token count after similarity search: 4110
Chain Type selected: map_reduce
CPU times: total: 93.8 ms
Wall time: 6.33 s


In [14]:
# Get the answer
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type)
response['output_text']

"Reinforcement learning has been used in many applications, including board games, opponent modeling in commercial games, Sokoban, verifying game design and playing strategies, ensemble pruning, and position evaluation in the game of Othello. It is also being increasingly used in routing. Reinforcement learning algorithms are designed to optimize long-term return by adjusting the behavior of an agent interacting with an environment, through a Markov decision process. The effectiveness of the agent's actions is communicated through a scalar value (reinforcement signal). \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf"

And if we ask the follow up question:

In [15]:
response = get_answer(docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English",deployment=MODEL, chain_type=chain_type)
response['output_text']

'For the first two documents, there is no relevant text to answer the question or rephrase the statement. The third document discusses the issue of finding a near-optimal policy in reinforcement learning, while the fourth document discusses the issue of estimating the value of policies in reinforcement learning without access to a generative model of the environment. Both documents focus on different techniques and algorithms to address these issues. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

In [16]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning is finding use in many important applications, including routing. Research in reinforcement learning focuses on designing algorithms for an agent interacting with an environment, to adjust its behavior in such a way as to optimize a long-term return. While there is no direct answer to the question in the given portion of the document, some examples of use cases for reinforcement learning can be found in literature. These include board games, opponent modelling and commercial games, single-agent search methods, verifying game design and playing strategies, position evaluation in the game of Othello, Markov games as a framework for multi-agent reinforcement learning, ensemble pruning, and practical issues in temporal difference learning. \n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [17]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize if my previous response was unclear. What I meant to say is that while the given portion of the document does not provide a direct answer to your question, some examples of use cases for reinforcement learning can be found in literature. These include board games, opponent modelling and commercial games, single-agent search methods, verifying game design and playing strategies, position evaluation in the game of Othello, Markov games as a framework for multi-agent reinforcement learning, ensemble pruning, and practical issues in temporal difference learning. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [18]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! Let me know if you have any other questions.\nSOURCES: N/A"

Let's check our memory to see that it's keeping the conversation

In [19]:
memory.buffer

"Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning is finding use in many important applications, including routing. Research in reinforcement learning focuses on designing algorithms for an agent interacting with an environment, to adjust its behavior in such a way as to optimize a long-term return. While there is no direct answer to the question in the given portion of the document, some examples of use cases for reinforcement learning can be found in literature. These include board games, opponent modelling and commercial games, single-agent search methods, verifying game design and playing strategies, position evaluation in the game of Othello, Markov games as a framework for multi-agent reinforcement learning, ensemble pruning, and practical issues in temporal difference learning. \n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://de

**Now, add memory in Azure CosmosDB:**

in previous cell we have added memory to provide conversation interface, now here we will store the conversation history into CosmosDB for future auditing purpose.

In [27]:
# Create CosmosDB instance from langchain cosmos class.

SESSION_ID = '001'  # this isstring value, normally you need to provide the user session id. As an example I have provided '001' as a session id
USER_ID = 'dummy01' # this is string value, normally you need to provide the user id. As an example I have provided 'dummy01' as a user id

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=AZURE_COSMOSDB_ENDPOINT,
    cosmos_database=AZURE_COSMOSDB_NAME,
    cosmos_container=AZURE_COSMOSDB_CONTAINER_NAME,
    connection_string=AZURE_COMOSDB_CONNECTION_STRING,
    session_id=SESSION_ID,
    user_id=USER_ID
    )

# prepare the cosmosdb instance

cosmos.prepare_cosmos()

2023-05-24 17:12:52.352 INFO    azure.core.pipeline.policies.http_logging_policy: Request URL: 'https://cosmosdb-account-onqlzkigt2nsi.documents.azure.com:443/'
Request method: 'GET'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-date': 'REDACTED'
    'authorization': 'REDACTED'
    'Accept': 'application/json'
    'Content-Length': '0'
    'User-Agent': 'azsdk-python-cosmos/4.3.1 Python/3.10.9 (Windows-10-10.0.22621-SP0)'
No body was attached to the request
2023-05-24 17:12:52.852 INFO    azure.core.pipeline.policies.http_logging_policy: Response status: 200
Response headers:
    'Cache-Control': 'no-store, no-cache'
    'Pragma': 'no-cache'
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Location': 'REDACTED'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'x-ms-max-media-storage-usage-mb': 'REDACTED'
    'x-ms-media-storage-usage-mb': 'REDACTED'


In [28]:

# load the conversation history into CosmosDB

memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

2023-05-24 17:13:12.297 INFO    azure.core.pipeline.policies.http_logging_policy: Request URL: 'https://cosmosdb-account-onqlzkigt2nsi-eastus.documents.azure.com:443/dbs/cosmosdb-db-onqlzkigt2nsi/colls/cosmosdb-container-onqlzkigt2nsi/'
Request method: 'GET'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-consistency-level': 'REDACTED'
    'x-ms-date': 'REDACTED'
    'authorization': 'REDACTED'
    'Accept': 'application/json'
    'Content-Length': '0'
    'User-Agent': 'azsdk-python-cosmos/4.3.1 Python/3.10.9 (Windows-10-10.0.22621-SP0)'
No body was attached to the request
2023-05-24 17:13:12.400 INFO    azure.core.pipeline.policies.http_logging_policy: Response status: 200
Response headers:
    'Cache-Control': 'no-store, no-cache'
    'Pragma': 'no-cache'
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Location': 'REDACTED'
    'Server': 'Mi

'Reinforcement learning has numerous use cases, such as in board games, ensemble pruning, and machine learning using checkers. Additionally, reinforcement learning is increasingly being used in important applications, including routing. The success of learning algorithms depends on the richness of information about various behaviors and how effectively it is used. Unfortunately, the given portion of the document does not provide a direct answer to your question. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [29]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

2023-05-24 17:13:39.492 INFO    azure.core.pipeline.policies.http_logging_policy: Request URL: 'https://cosmosdb-account-onqlzkigt2nsi-eastus.documents.azure.com:443/dbs/cosmosdb-db-onqlzkigt2nsi/colls/cosmosdb-container-onqlzkigt2nsi/docs/'
Request method: 'POST'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-consistency-level': 'REDACTED'
    'x-ms-session-token': 'REDACTED'
    'x-ms-documentdb-partitionkey': 'REDACTED'
    'x-ms-date': 'REDACTED'
    'authorization': 'REDACTED'
    'Content-Type': 'application/json'
    'Accept': 'application/json'
    'x-ms-documentdb-is-upsert': 'REDACTED'
    'Content-Length': '1053'
    'User-Agent': 'azsdk-python-cosmos/4.3.1 Python/3.10.9 (Windows-10-10.0.22621-SP0)'
A body is sent with the request
2023-05-24 17:13:39.597 INFO    azure.core.pipeline.policies.http_logging_policy: Response status: 200
Response headers:
    'Cache-Control': 

'I apologize for any confusion. What I meant to say is that reinforcement learning has various applications, including board games, ensemble pruning, and machine learning with checkers. In addition, reinforcement learning is increasingly being used in important applications such as routing. However, the given portion of the documents does not provide a direct answer to your question. Can you please provide more context or a specific question? \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [30]:
response

{'output_text': 'I apologize for any confusion. What I meant to say is that reinforcement learning has various applications, including board games, ensemble pruning, and machine learning with checkers. In addition, reinforcement learning is increasingly being used in important applications such as routing. However, the given portion of the documents does not provide a direct answer to your question. Can you please provide more context or a specific question? \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'}

In [31]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

2023-05-24 17:13:51.850 INFO    azure.core.pipeline.policies.http_logging_policy: Request URL: 'https://cosmosdb-account-onqlzkigt2nsi-eastus.documents.azure.com:443/dbs/cosmosdb-db-onqlzkigt2nsi/colls/cosmosdb-container-onqlzkigt2nsi/docs/'
Request method: 'POST'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-consistency-level': 'REDACTED'
    'x-ms-session-token': 'REDACTED'
    'x-ms-documentdb-partitionkey': 'REDACTED'
    'x-ms-date': 'REDACTED'
    'authorization': 'REDACTED'
    'Content-Type': 'application/json'
    'Accept': 'application/json'
    'x-ms-documentdb-is-upsert': 'REDACTED'
    'Content-Length': '1886'
    'User-Agent': 'azsdk-python-cosmos/4.3.1 Python/3.10.9 (Windows-10-10.0.22621-SP0)'
A body is sent with the request
2023-05-24 17:13:52.104 INFO    azure.core.pipeline.policies.http_logging_policy: Response status: 200
Response headers:
    'Cache-Control': 

"You're welcome! Let me know if you have any other questions or if there's anything else I can help you with.\nSOURCES: N/A"

Let's check our Azure CosmosDB to see the whole conversation


In [34]:
#load message from cosmosdb

cosmos.load_messages()
cosmos.messages

2023-05-24 17:22:12.394 INFO    azure.core.pipeline.policies.http_logging_policy: Request URL: 'https://cosmosdb-account-onqlzkigt2nsi-eastus.documents.azure.com:443/dbs/cosmosdb-db-onqlzkigt2nsi/colls/cosmosdb-container-onqlzkigt2nsi/docs/001/'
Request method: 'GET'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-consistency-level': 'REDACTED'
    'x-ms-session-token': 'REDACTED'
    'x-ms-documentdb-partitionkey': 'REDACTED'
    'x-ms-date': 'REDACTED'
    'authorization': 'REDACTED'
    'Accept': 'application/json'
    'Content-Length': '0'
    'User-Agent': 'azsdk-python-cosmos/4.3.1 Python/3.10.9 (Windows-10-10.0.22621-SP0)'
No body was attached to the request
2023-05-24 17:22:12.851 INFO    azure.core.pipeline.policies.http_logging_policy: Response status: 200
Response headers:
    'Cache-Control': 'no-store, no-cache'
    'Pragma': 'no-cache'
    'Transfer-Encoding': 'chunked

[HumanMessage(content='Tell me some use cases for reinforcement learning?', additional_kwargs={}, example=False),
 AIMessage(content='Reinforcement learning has numerous use cases, such as in board games, ensemble pruning, and machine learning using checkers. Additionally, reinforcement learning is increasingly being used in important applications, including routing. The success of learning algorithms depends on the richness of information about various behaviors and how effectively it is used. Unfortunately, the given portion of the document does not provide a direct answer to your question. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf', additional_kwargs={}, example=False),
 HumanMessage(content='Can you rephrase what you just said?', additional_kwargs={}, example=False),
 AIMessage(content='I apologize fo

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base



# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook 05 explains and solves the tabular problem and the concept of Agents